In [1]:
from collections import OrderedDict
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from base import BaseModel

In [2]:
# Link Doesn't work!
model_paths = {
    'darknet19': 'https://s3.ap-northeast-2.amazonaws.com/deepbaksuvision/darknet19-deepBakSu-e1b3ec1e.pth'
}

In [3]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        N = x.data.size(0)
        C = x.data.size(1)
        H = x.data.size(2)
        W = x.data.size(3)
        x = F.avg_pool2d(x, (H, W))
        x = x.view(N, C)
        return x

In [4]:
class Darknet19(BaseModel):
    def __init__(self, pretrained=True):
        super(Darknet19, self).__init__()

        self.features = nn.Sequential(OrderedDict([
        ('layer1', nn.Sequential(OrderedDict([
            ('conv1_1', nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn1_1', nn.BatchNorm2d(32)),
            ('leaky1_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool1', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer2', nn.Sequential(OrderedDict([
            ('conv2_1', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn2_1', nn.BatchNorm2d(64)),
            ('leaky2_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool2', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer3', nn.Sequential(OrderedDict([
            ('conv3_1', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_1', nn.BatchNorm2d(128)),
            ('leaky3_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_2', nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn3_2', nn.BatchNorm2d(64)),
            ('leaky3_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_3', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_3', nn.BatchNorm2d(128)),
            ('leaky3_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool3', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer4', nn.Sequential(OrderedDict([
            ('conv4_1', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_1', nn.BatchNorm2d(256)),
            ('leaky4_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_2', nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn4_2', nn.BatchNorm2d(128)),
            ('leaky4_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_3', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_3', nn.BatchNorm2d(256)),
            ('leaky4_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool4', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer5', nn.Sequential(OrderedDict([
            ('conv5_1', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_1', nn.BatchNorm2d(512)),
            ('leaky5_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_2', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn5_2', nn.BatchNorm2d(256)),
            ('leaky5_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_3', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_3', nn.BatchNorm2d(512)),
            ('leaky5_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_4', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn5_4', nn.BatchNorm2d(256)),
            ('leaky5_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_5', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_5', nn.BatchNorm2d(512)),
            ('leaky5_5', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool5', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer6', nn.Sequential(OrderedDict([
            ('conv6_1', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_1', nn.BatchNorm2d(1024)),
            ('leaky6_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_2', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn6_2', nn.BatchNorm2d(512)),
            ('leaky6_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_3', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_3', nn.BatchNorm2d(1024)),
            ('leaky6_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_4', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn6_4', nn.BatchNorm2d(512)),
            ('leaky6_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_5', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_5', nn.BatchNorm2d(1024)),
            ('leaky6_5', nn.LeakyReLU(0.1, inplace=True))
        ])))
        ]))

        self.classifier = nn.Sequential(OrderedDict([
        ('conv7_1', nn.Conv2d(1024, 1000, kernel_size=(1, 1), stride=(1, 1))),
        ('globalavgpool', GlobalAvgPool2d()),
        ('softmax', nn.Softmax(dim=1))
        ]))

        if pretrained:
            # self.load_state_dict(model_zoo.load_url(model_paths['darknet19'],  progress=True))
            self.load_weight()
            print('Model is loaded')

    def forward(self, x):
        out = self.features(x)
        out = self.classifier(out)
        return out
    def load_weight(self):
        weight_file = 'weights/darknet19-deepBakSu-e1b3ec1e.pth'
        assert len(torch.load(weight_file).keys()) == len(self.state_dict().keys())
        dic = {}
        for now_keys, values in zip(self.state_dict().keys(), torch.load(weight_file).values()):
            dic[now_keys]=values
        self.load_state_dict(dic)
        print('Weights are loaded!')

In [5]:
model = Darknet19(True)

Weights are loaded!
Model is loaded


In [6]:
# Block 1's Convolutional layer weights.
firstWeightMatrix = model.features[0][0].weight
firstBiasMatrix = model.features[0][0].bias

In [7]:
sum(p.numel() for p in model.features[0][0].parameters())

864

In [8]:
firstBiasMatrix # Bias is set to false so no bias.

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [10]:
count_parameters(model)

20842376

In [11]:
import numpy as np        

weights = open('conv_1_1.csv', 'ab')

In [12]:
np.savetxt('conv_1_1.csv', firstWeightMatrix.detach().numpy().ravel())

In [18]:
model.features[0][0].out_channels

32

In [15]:
a = model.features[0][1]

In [20]:
a.running_mean.detach().numpy().ravel()

array([-2.3214601e-03, -2.5296986e-01,  1.1893848e-03,  2.3869975e-03,
        2.2873398e-02, -1.3856249e-03, -3.0809397e-01,  1.1708887e-01,
       -3.0102493e-04, -1.4464438e-02, -4.5391001e-02,  1.8429115e-01,
       -2.8679994e-01,  1.9463764e-01,  6.2633986e-03,  2.0220485e-03,
        5.5335030e-02,  3.7532397e-02, -1.8442741e-02, -3.1703014e-03,
        4.7915992e-03,  3.3819050e-02, -2.2025496e-01, -3.4106191e-02,
        1.0814303e-01,  5.4634456e-02, -2.0843312e-01, -5.0900027e-04,
       -1.7058287e-02, -1.9609613e-02,  2.8884465e-01, -4.7691960e-02],
      dtype=float32)

In [28]:
if isinstance(a, nn.Conv2d):
    name = "Convolution"

In [29]:
name

'Convolution'

In [39]:
model.features[0][1].weight

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False])

In [40]:
model.features[0][1].bias

Parameter containing:
tensor([ -1.9168,  -1.7893,   0.8074,   2.1464,   1.9664,   0.7599,  -6.7155,
         -3.2311,  -1.1955,  -2.3681,  -2.9968,  -6.5154, -10.2869,  -2.1426,
          0.8042,   0.4698,   1.0785,   0.8214,   0.4473,  -2.4476,   0.6459,
          0.7412,  -4.4340,   3.2402,  -1.3607,   1.3372,  -1.8851,  -3.2254,
         -2.5978,   0.3849,  -7.8350,  -0.0251], requires_grad=True)

In [42]:
model.features[0][1].bias.size()

torch.Size([32])

In [43]:
model.features[0][1].weight.size()

torch.Size([32])

AttributeError: 'Conv2d' object has no attribute 'input_channel'